In [11]:
!pip install langchain_community pypdf langchain

  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
   ---------------------------------------- 0.0/817.7 kB ? eta -:--:--
    --------------------------------------- 10.2/817.7 kB ? eta -:--:--
   --- ----------------------------------- 71.7/817.7 kB 975.2 kB/s eta 0:00:01
   ----------------- ---------------------- 358.4/817.7 kB 2.8 MB/s eta 0:00:01
   ------------------------------ --------- 624.6/817.7 kB 3.6 MB/s eta 0:00:01
   ---------------------------------- ----- 696.3/817.7 kB 3.6 MB/s eta 0:00:01
   ------------------------------------ --- 737.3/817.7 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 817.7/817.7 kB 2.9 MB/s eta 0:00:00
Using cached langchain_text_splitters-0.0.1-py3-none-any.whl (21 kB)


In [3]:
question = "can you summarize the benefits of apple cider vinegar"

In [4]:
from langchain_community.document_loaders import PyPDFLoader

In [13]:
loader = PyPDFLoader("documents/apple_cider.pdf")

In [14]:
pages = loader.load_and_split()


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=20)

In [17]:
documents = text_splitter.split_documents(pages)

In [18]:
print(f"{len(pages)} vs {len(documents)}")

17 vs 57


In [ ]:
!pip install dotenv

In [20]:
import os 
from dotenv import load_dotenv

load_dotenv(".env")

openai_api_key = os.getenv("open_ai_key")

In [21]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [22]:
from langchain_community.vectorstores import FAISS

In [24]:
!pip install faiss-cpu

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.5 MB 1.3 MB/s eta 0:00:11
    --------------------------------------- 0.3/14.5 MB 2.8 MB/s eta 0:00:06
   - -------------------------------------- 0.6/14.5 MB 3.5 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/14.5 MB 4.0 MB/s eta 0:00:04
   -- ------------------------------------- 1.0/14.5 MB 4.1 MB/s eta 0:00:04
   ---- ----------------------------------- 1.5/14.5 MB 5.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/14.5 MB 5.3 MB/s eta 0:00:03
   ----- ---------------------------------- 2.1/14.5 MB 5.4 MB/s eta 0:00:03
   ------ --------------------------------- 2.4/14.5 MB 5.6 MB/s eta 0:00:03
   ------- -------------------------------- 2.7/14.5 MB 5.6 MB/s eta 0:00:03
   -------- ------------------------------- 3.0/14.5 MB 5.6 MB/s eta 0:00:03
   --------- 

In [25]:
vector = FAISS.from_documents(documents, embeddings)

In [26]:
vector.similarity_search_with_score(question)

[(Document(page_content='the necessity of discerning novel and efficacious ap-\nproaches. In this line, the benefical effect of nutraceutics\nand fuctional foods on human health have been well-\ndocumented [ 8,36]. In contemporary research and prac-\ntice, plants and their derivatives have attracted a lot of\ninterest for their beneficial effects in controlling lipid\nprofile and glycemic status [ 5,15,37]. Indeed, one of the\nmost popular plant derivatives in this regard is vinegar.\nApple cider vinegar (ACV) is one of the three most\ncommon types of vinegar, produced by fermenting ap-\nples [ 3]. This acidic solution is consumed throughout\nthe world as a flavoring and preservative agent in foods\n[22]. ACV contains a variety of flavonoids, such as gallic\nacid, catechin, caffeic acid, and ferulic acid [ 11,31]. Ani-\nmal experiments have reported that ACV has a variety\nof pharmacological functions, including anti-oxidant,\nanti-inflammatory, anti-diabetic, anti-hypertensive, and', 

In [27]:
from langchain_core.prompts import ChatPromptTemplate

prompt= ChatPromptTemplate.from_template("""
Answer the following questions based ont he provided context.
    If the context did not answer the question do the followings: 
    1. Mention that you could not find the exact answer, 
    2. provide a summary of the contxt
    
Context:                          
{context}

Question: 
{question}   

Your response:           
                           
                           """)

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")

In [29]:
from langchain_core.output_parsers import StrOutputParser
output_parse = StrOutputParser()

In [30]:
retriever = vector.as_retriever()


In [34]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_and_retrieval = RunnableParallel(
    {"context":retriever, "question": RunnablePassthrough()}
)

In [35]:
chain = setup_and_retrieval | prompt | llm | output_parse

In [36]:
response = chain.invoke(question)

In [ ]:
print(response)